<a href="https://colab.research.google.com/github/vellamike/pycuda/blob/master/pycuda_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to CUDA and PyCUDA

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
import numpy
numpy.random.seed(1729)
a = numpy.random.randn(4,4)

In [ ]:
a = a.astype(numpy.float32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [ ]:
cuda.memcpy_htod(a_gpu, a)


In [ ]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [ ]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [ ]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

In [ ]:
b = numpy.random.randn(4,4)
b = b.astype(numpy.float32)
c = numpy.random.randn(4,4)
c = c.astype(numpy.float32)

In [ ]:
mod2 = SourceModule("""
  __global__ void add2(float *a, float *b)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] += b[idx];
  }
  """)

In [ ]:
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

cuda.memcpy_htod(b_gpu, b)
cuda.memcpy_htod(c_gpu, c)


In [ ]:
func = mod2.get_function("add2")
func(b_gpu,c_gpu, block=(4,4,1))

In [ ]:
added = numpy.empty_like(b)
cuda.memcpy_dtoh(added, b_gpu)
print(added)
print(b)
print(c)

# Exercises

1. Write a cuda kernel to find the elementwise square of a matrix
2. Write a cuda kernel to find a matrix, which when added to the given matrix results in every element being equal to zero
3. Write a cuda kernel to multiply two matrices:
    1. Assume square matrices, with dimensions < 1024
    2. Assume square matrices, with dimensions > 1024
    3. Assume non-square matrices, with dimensions > 1024

**Question 1**

In [ ]:
mod3 = SourceModule("""
  __global__ void square2(float *a, int size)
  {
    int idx = threadIdx.x + threadIdx.y*size;
    a[idx] *= a[idx];
  }
  """)

In [ ]:
size = 7

numpy.random.seed(945)
d = numpy.random.randn(size,size)
d = d.astype(numpy.float32)

In [ ]:
d_gpu = cuda.mem_alloc(d.nbytes)
cuda.memcpy_htod(d_gpu, d)

In [ ]:
func = mod3.get_function("square2")
func(d_gpu, numpy.int32(size), block=(size,size,1))

In [ ]:
squared = numpy.empty_like(d)
cuda.memcpy_dtoh(squared, d_gpu)

In [ ]:
#print(d)
#print(squared)

print(numpy.max(numpy.abs(squared - d*d)))

**Question 2**

In [ ]:
mod4 = SourceModule("""
  __global__ void minus(float *a, int size)
  {
    int idx = threadIdx.x + threadIdx.y*size;
    a[idx] = -a[idx];
  }
  """)

In [ ]:
size = 7

numpy.random.seed(9574)
d = numpy.random.randn(size,size)
d = d.astype(numpy.float32)

In [ ]:
d_gpu = cuda.mem_alloc(d.nbytes)
cuda.memcpy_htod(d_gpu, d)

In [ ]:
func = mod4.get_function("minus")
func(d_gpu, numpy.int32(size), block=(size,size,1))

In [ ]:
complement = numpy.empty_like(d)
cuda.memcpy_dtoh(complement, d_gpu)

In [ ]:
#print(d)
#print(complement)

print(numpy.max(numpy.abs(d + complement)))

**Question 3**

   A. Assume square matrices, with dimensions < 1024

In [ ]:
mod5 = SourceModule("""
  __global__ void multiplyA(float *a, float *b, float *c, int size)
  {
    int row = threadIdx.x; 
    int col = threadIdx.y;
    
    c[col + row * size] = 0;
    for(int i=0; i<size; ++i){
        c[col + row * size] += a[i + row * size] * b[col + i * size];
    }
  }
  """)

In [ ]:
size = 8

numpy.random.seed(9574)

a = numpy.random.randn(size,size)
a = a.astype(numpy.float32)

b = numpy.random.randn(size,size)
b = b.astype(numpy.float32)

c = numpy.empty_like(a)
c = c.astype(numpy.float32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)

c_gpu = cuda.mem_alloc(c.nbytes)

In [ ]:
func = mod5.get_function("multiplyA")
func(a_gpu, b_gpu, c_gpu, numpy.int32(size), block=(size,size,1))

In [ ]:
cuda.memcpy_dtoh(c, c_gpu)

In [ ]:
#print(a@b)
#print(c)

print(numpy.max(numpy.abs(c - a@b)))

B. Assume square matrices, with dimensions > 1024

In [ ]:
mod6 = SourceModule("""
  __global__ void multiplyB(float *a, float *b, float *c, int size)
  {
    int row = threadIdx.x + blockDim.x * blockIdx.x; 
    int col = threadIdx.y + blockDim.y * blockIdx.y;
    
    if (row < size && col < size){
        c[col + row * size] = 0;
        for(int i=0; i<size; ++i){
            c[col + row * size] += a[i + row * size] * b[col + i * size];
        } 
    }
  }
  """)

In [ ]:
size = 56

numpy.random.seed(494)

a = numpy.random.randn(size,size)
a = a.astype(numpy.float32)

b = numpy.random.randn(size,size)
b = b.astype(numpy.float32)

c = numpy.empty_like(a)
c = c.astype(numpy.float32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)

c_gpu = cuda.mem_alloc(c.nbytes)

In [ ]:
block_size = 32
grid_size = (size + block_size - 1) // block_size # Number of needed blocks to cover the whole matrix

In [ ]:
func = mod6.get_function("multiplyB")
func(a_gpu, b_gpu, c_gpu, numpy.int32(size), block=(block_size,block_size,1), grid=(grid_size,grid_size,1))

In [ ]:
cuda.memcpy_dtoh(c, c_gpu)

In [ ]:
#print(a@b)
#print(c)

print(numpy.max(numpy.abs(c - a@b)))

C. Assume non-square matrices, with dimensions > 1024

In [ ]:
mod7 = SourceModule("""
  __global__ void multiplyC(float *a, float *b, float *c, int n, int m, int p)
  {
    int row = threadIdx.x + blockDim.x * blockIdx.x; 
    int col = threadIdx.y + blockDim.y * blockIdx.y;
    
    if (row < n && col < m){
        c[col + row * m] = 0;
        for(int i=0; i<p; ++i){
            c[col + row * m] += a[i + row * p] * b[col + i * m];
        } 
    }
  }
  """)

In [ ]:
n = 36
p = 48
m = 72

numpy.random.seed(5867)

a = numpy.random.randn(n,p)
a = a.astype(numpy.float32)

b = numpy.random.randn(p,m)
b = b.astype(numpy.float32)

c = numpy.zeros([n,m])
c = c.astype(numpy.float32)

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu, b)

c_gpu = cuda.mem_alloc(c.nbytes)

In [ ]:
block_size_x = min(n, 32) 
grid_size_x = (n + block_size_x - 1) // block_size_x

block_size_y = min(m, 32)
grid_size_y = (m + block_size_y - 1) // block_size_y

In [ ]:
func = mod7.get_function("multiplyC")
func(a_gpu, b_gpu, c_gpu, 
     numpy.int32(n), numpy.int32(m), numpy.int32(p), 
     block=(block_size_x,block_size_y,1), grid=(grid_size_x,grid_size_y,1))

In [ ]:
cuda.memcpy_dtoh(c, c_gpu)

In [ ]:
#print(a@b)
#print(c)

print(numpy.max(numpy.abs(c - a@b)))